<a href="https://colab.research.google.com/github/bkkaggle/pytorch-CycleGAN-and-pix2pix/blob/master/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

# Install

In [7]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...


In [8]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [9]:
!pip install -r requirements.txt

     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ------------ --------------------------- 0.4/1.4 MB 6.9 MB/s eta 0:00:01
     ---------------------------------------- 1.4/1.4 MB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------- ----- 1.9/2.2 MB 60.4 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 46.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/195.4 kB ? eta -:--:--
   ---------------------------------------- 195.4/195.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/258.5 kB ? eta -:--:--
   --------------------------------------- 258.5/258.5 kB 16.5 MB/s eta 0:00:00
Using cached fs

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_cyclegan_dataset.sh [apple2orange, summer2winter_yosemite, horse2zebra, monet2photo, cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades, iphone2dslr_flower, ae_photos]`

Or use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

In [16]:
#download the horse2zebra dataset
!bash ./datasets/download_cyclegan_dataset.sh horse2zebra

'bash' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_cyclegan_model.sh [apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [11]:
!bash ./scripts/download_cyclegan_model.sh horse2zebra

'bash' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


In [12]:
!python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan --display_id -1

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./datasets/horse2zebra        	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            	[default: 1]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
 

Traceback (most recent call last):
  File "train.py", line 29, in <module>
    dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
  File "C:\Users\leoch\Desktop\comp8501 A1\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix\data\__init__.py", line 57, in create_dataset
    data_loader = CustomDatasetDataLoader(opt)
  File "C:\Users\leoch\Desktop\comp8501 A1\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix\data\__init__.py", line 73, in __init__
    self.dataset = dataset_class(opt)
  File "C:\Users\leoch\Desktop\comp8501 A1\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix\data\unaligned_dataset.py", line 29, in __init__
    self.A_paths = sorted(make_dataset(self.dir_A, opt.max_dataset_size))   # load images from '/path/to/data/trainA'
  File "C:\Users\leoch\Desktop\comp8501 A1\pytorch-CycleGAN-

# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [13]:
!python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: datasets/horse2zebra/testA    	[default: None]
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

Traceback (most recent call last):
  File "test.py", line 50, in <module>
    dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
  File "C:\Users\leoch\Desktop\comp8501 A1\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix\data\__init__.py", line 57, in create_dataset
    data_loader = CustomDatasetDataLoader(opt)
  File "C:\Users\leoch\Desktop\comp8501 A1\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix\data\__init__.py", line 73, in __init__
    self.dataset = dataset_class(opt)
  File "C:\Users\leoch\Desktop\comp8501 A1\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pix\data\single_dataset.py", line 19, in __init__
    self.A_paths = sorted(make_dataset(opt.dataroot, opt.max_dataset_size))
  File "C:\Users\leoch\Desktop\comp8501 A1\pytorch-CycleGAN-and-pix2pix-master\pytorch-CycleGAN-and-pix2pi

# Visualize

In [14]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_fake.png')
plt.imshow(img)

FileNotFoundError: [Errno 2] No such file or directory: './results/horse2zebra_pretrained/test_latest/images/n02381460_1010_fake.png'

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_real.png')
plt.imshow(img)